<a href="https://colab.research.google.com/github/rizzlogy/RealESRGAN-GColab/blob/main/RealESRGAN_GColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**RealESRGAN-GColab**
==
**RealESRGAN-GColab** leverages the power of **Nvidia T4 GPU** available in **Google Colab** to **enhance image quality with sharper details and improved clarity automatically**. By utilizing the Nvidia T4 GPU, **RealESRGAN-GColab** is capable of delivering superior and **faster results** in enhancing image quality within the **Google Colab** environment.

In [ ]:
#@title **Install Package**

!pip install git+https://github.com/sberbank-ai/Real-ESRGAN.git

from IPython.display import clear_output
clear_output()
print("Success To Install package")

In [ ]:
#@title **Select Scale**
from RealESRGAN import RealESRGAN
from IPython.display import clear_output
import numpy as np
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

model_scale = "4" #@param ["2", "4", "8"] {allow-input: false}

try:
    model = RealESRGAN(device, scale=int(model_scale))
    model.load_weights(f'weights/RealESRGAN_x{model_scale}.pth')
    clear_output()
    print(f"Model Scale {model_scale} Has Been Installed!")
except Exception as e:
    clear_output()
    print('Something Is Wrong..')
    print(e)

In [ ]:
#@title **Upload Photo**
import os
import sys
from google.colab import files
import shutil
from io import BytesIO
import io
import tarfile
from PIL import Image
import numpy as np

upload_folder = 'inputs'
result_folder = 'results'

os.makedirs(upload_folder, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)

IMAGE_FORMATS = ('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')

def image_to_tar_format(img, image_name):
    buff = BytesIO()
    if '.png' in image_name.lower():
        img = img.convert('RGBA')
        img.save(buff, format='PNG')
    else:
        img.save(buff, format='JPEG')
    buff.seek(0)
    fp = io.BufferedReader(buff)
    img_tar_info = tarfile.TarInfo(name=image_name)
    img_tar_info.size = len(buff.getvalue())
    return img_tar_info, fp

def process_tar(path_to_tar):
    processing_tar = tarfile.open(path_to_tar, mode='r')
    result_tar_path = os.path.join('results/', os.path.basename(path_to_tar))
    save_tar = tarfile.open(result_tar_path, 'w')

    for c, member in enumerate(processing_tar):
        print(f'{c}, Processing {member.name}')

        if not member.name.endswith(IMAGE_FORMATS):
            continue

        try:
            img_bytes = BytesIO(processing_tar.extractfile(member.name).read())
            img_lr = Image.open(img_bytes, mode='r').convert('RGB')
        except Exception as err:
            print(f'Unable to open file {member.name}, skipping')
            continue

        img_sr = model.predict(np.array(img_lr))
        # adding to save_tar
        img_tar_info, fp = image_to_tar_format(img_sr, member.name)
        save_tar.addfile(img_tar_info, fp)

    processing_tar.close()
    save_tar.close()
    print(f'Finished! Archive saved to {result_tar_path}')

def process_input(filename):
    if tarfile.is_tarfile(filename):
        process_tar(filename)
    else:
        result_image_path = os.path.join('results/', os.path.basename('[HD] ' + filename))
        image = Image.open(filename).convert('RGB')
        sr_image = model.predict(np.array(image))
        sr_image.save(result_image_path)
        print(f'Finished!\n')

        # Remove the original and processed files
        os.remove(filename)

# Upload files
try:
    print('Select File To Enhanced\n')
    uploaded = files.upload()
    for filename in uploaded.keys():
        print('\nEnhance The Image:', filename)
        process_input(filename)
        # Zip the results folder
        shutil.make_archive('download', 'zip', 'results')
        # Download the zip file
        files.download('download.zip')
except KeyboardInterrupt:
    print('\nCanceled upload.')
    sys.exit()
finally:
    # Remove the upload & results folder
    shutil.rmtree(upload_folder)
    shutil.rmtree(result_folder)

# DO NOT CHANGE THIS CODE OR YOU WILL GET AN ERROR!!
# CODE BY RIZZYFUZZ & FASTURL LLC.

#### **© RIZZYFUZZ & FASTURL.CLOUD**